#### 朴素贝叶斯算法  
**输入**：训练数据$T=\{(x_1,y_1),(x_2,y_2),...,(x_N,y_N) \}$，其中$x_i=(x_i^{(1)},x_i^{(2)},...,x_i^{(n)})$，$x_i^{(j)}$是第$i$个样本的第$j$个特征，$x_i^{(j)}\in \{a_{j1},a_{j2},...,a_{jS_j} \}$，  
&emsp;$a_{jl}$是第$j$个特征可能取的第$l$个值，$j=1,2,...,S_j$，$y_i\in c_1,c_2,...c_K$；实例$x$；  
**输出**：实例$x$的分类。  
（1）计算先验概率及条件概率  
&emsp;&emsp;&emsp;$P(Y=c_k)=\frac{\sum_{i=1}^NI(y_i=c_k)}{N},k=1,2,...,K $  
&emsp;&emsp;&emsp;$P(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^NI(x_i^{(j)}=a_{jl},y_i=c_k)}{\sum_{i=1}^NI(y_i=c_k)} $  
&emsp;&emsp;&emsp;$j=1,2,...,n;l=1,2,...,S;k=1,2,...,K$  
（2）对于给定的实例$x=(x^{(1)},x^{(2)},...,x^{(n)})^T$，计算  
&emsp;&emsp;&emsp;$P(Y=c_k)\prod_{j=1}^{n}P(X^{(j)}=x^{(j)}|Y=c_k),k=1,2,...,K $  
（3）确定实例$x$的类  
&emsp;&emsp;&emsp;$y=arg\max_{c_k}P(Y=c_k)\prod_{j=1}^{n}P(X^{(j)}=x^{(j)}|Y=c_k) $

In [1]:
import numpy as np

#### 加载文件  
$fileName$: 要加载的文件路径  
$return$: 数据集和标签集

In [2]:
def loadData(fileName):
    dataArr = []
    labelArr = []
    fr = open(fileName)
    #遍历文件中的每一行
    for line in fr.readlines():
        curLine = line.strip().split(',')
        dataArr.append([int(int(num) > 128) for num in curLine[1:]])
        labelArr.append(int(curLine[0]))
    #返回数据集和标记
    return dataArr, labelArr

#### 通过朴素贝叶斯进行概率估计  
$P_y$: 先验概率分布  
$P_{x\_y}$: 条件概率分布  
$x$: 要估计的样本x  
*return*: 返回所有label的估计概率

In [3]:
def NaiveBayes(Py, Px_y, x):
    #设置特征数目，类别数目
    featrueNum = 784
    classNum = 10
    P = [0] * classNum
    #对于每一个类别，单独估计其概率
    for i in range(classNum):
        sum = 0
        #获取每一个条件概率值，进行累加
        for j in range(featrueNum):
            sum += Px_y[i][j][x[j]]
        P[i] = sum + Py[i]

    #max(P)：找到概率最大值
    return P.index(max(P))

#### 对测试集进行测试  
$P_y$: 先验概率分布  
$P_{x\_y}$: 条件概率分布  
$testDataArr$: 测试集数据  
$testLabelArr$: 测试集标记  
*return*: 准确率

In [4]:
def model_test(Py, Px_y, testDataArr, testLabelArr):
    #错误值计数
    errorCnt = 0
    #循环遍历测试集中的每一个样本
    for i in range(len(testDataArr)):
        presict = NaiveBayes(Py, Px_y, testDataArr[i])
        #与答案进行比较
        if presict != testLabelArr[i]:
            errorCnt += 1
    #返回准确率
    return 1 - (errorCnt / len(testDataArr))

#### 通过训练集计算先验概率分布和条件概率分布  
$trainDataArr$: 训练数据集  
$trainLabelArr$: 训练标记集  
*return*: 先验概率分布和条件概率分布

In [5]:
def getAllProbability(trainDataArr, trainLabelArr):
    #设置样本特诊数目，数据集中手写图片为28*28，转换为向量是784维。
    featureNum = 784
    classNum = 10
    #初始化先验概率分布存放数组，后续计算得到的P(Y = 0)放在Py[0]中，以此类推
    Py = np.zeros((classNum, 1))
    #对每个类别进行一次循环，分别计算它们的先验概率分布
    for i in range(classNum):
        Py[i] = ((np.sum(np.mat(trainLabelArr) == i)) + 1) / (len(trainLabelArr) + 10)
    #转换为log对数形式
    Py = np.log(Py)
    Px_y = np.zeros((classNum, featureNum, 2))
    #对标记集进行遍历
    for i in range(len(trainLabelArr)):
        label = trainLabelArr[i]
        x = trainDataArr[i]
        for j in range(featureNum):
            Px_y[label][j][x[j]] += 1

    #循环每一个标记（共10个）
    for label in range(classNum):
        #循环每一个标记对应的每一个特征
        for j in range(featureNum):
            Px_y0 = Px_y[label][j][0]
            Px_y1 = Px_y[label][j][1]
            Px_y[label][j][0] = np.log((Px_y0 + 1) / (Px_y0 + Px_y1 + 2))
            Px_y[label][j][1] = np.log((Px_y1 + 1) / (Px_y0 + Px_y1 + 2))

    #返回先验概率分布和条件概率分布
    return Py, Px_y

#### 开始实验

In [6]:
# 获取训练集
print('start read trainSet')
trainDataArr, trainLabelArr = loadData('Mnist/mnist_train.csv')

# 获取测试集
print('start read testSet')
testDataArr, testLabelArr = loadData('Mnist/mnist_test.csv')

#开始训练，学习先验概率分布和条件概率分布
print('start to train')
Py, Px_y = getAllProbability(trainDataArr, trainLabelArr)

#使用习得的先验概率分布和条件概率分布对测试集进行测试
print('start to test')
accuracy = model_test(Py, Px_y, testDataArr, testLabelArr)

#打印准确率
print('the accuracy is:', accuracy)

start read trainSet
start read testSet
start to train
start to test
the accuracy is: 0.8432999999999999
